In [1]:
import pandas as pd
import numpy as np
import pickle


import sys
sys.path.insert(0, "../src")
sys.path.insert(0, "../")
import shutil
from d2v_recommender import *
from config import config

from autosklearn.classification import AutoSklearnClassifier
from autosklearn.metrics import roc_auc
import lightgbm
import sklearn

In [2]:
d2v = D2V_Recommender()
data_dict = d2v.load_data_dict(config.data_dict_path)
print(data_dict["X_train"].shape, data_dict["y_train"].shape, data_dict["X_test"].shape, data_dict["y_test"].shape)
X_train = data_dict["X_train"]
X_test = data_dict["X_test"]
y_train = data_dict["y_train"]
y_test = data_dict["y_test"]

(1883637, 200) (1883637,) (209119, 200) (209119,)


In [3]:
train_data = lightgbm.Dataset(X_train, label=y_train)
test_data = lightgbm.Dataset(X_test, label=y_test)

In [4]:
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 1
}

model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

16
[92]	valid_0's auc: 0.643165
[93]	valid_0's auc: 0.643247
[94]	valid_0's auc: 0.643367
[95]	valid_0's auc: 0.643396
[96]	valid_0's auc: 0.64341
[97]	valid_0's auc: 0.643415
[98]	valid_0's auc: 0.643484
[99]	valid_0's auc: 0.643564
[100]	valid_0's auc: 0.643646
[101]	valid_0's auc: 0.643745
[102]	valid_0's auc: 0.643832
[103]	valid_0's auc: 0.643895
[104]	valid_0's auc: 0.644014
[105]	valid_0's auc: 0.644102
[106]	valid_0's auc: 0.644081
[107]	valid_0's auc: 0.644092
[108]	valid_0's auc: 0.644094
[109]	valid_0's auc: 0.644185
[110]	valid_0's auc: 0.644176
[111]	valid_0's auc: 0.644233
[112]	valid_0's auc: 0.644209
[113]	valid_0's auc: 0.644183
[114]	valid_0's auc: 0.644258
[115]	valid_0's auc: 0.644338
[116]	valid_0's auc: 0.644424
[117]	valid_0's auc: 0.644467
[118]	valid_0's auc: 0.644493
[119]	valid_0's auc: 0.64452
[120]	valid_0's auc: 0.644571
[121]	valid_0's auc: 0.644552
[122]	valid_0's auc: 0.644529
[123]	valid_0's auc: 0.644597
[124]	valid_0's auc: 0.644642
[125]	valid_0's a

In [ ]:
# # Auto-ML

# shutil.rmtree(config.output_automl_path, ignore_errors=True)
# shutil.rmtree(config.tmp_automl_path, ignore_errors=True)
# automl = AutoSklearnClassifier(
#     time_left_for_this_task=180,
#     per_run_time_limit=60,
#     tmp_folder=config.tmp_automl_path,  # automated
#     output_folder=config.output_automl_path,
#     # metric=roc_auc
# )

# automl.fit(X_train, 
#             y_train,
#             dataset_name='d2v_small')

# ############################################################################
# # Print the final ensemble constructed by auto-sklearn
# # ====================================================

# print(automl.show_models())

In [6]:
from sklearn.metrics import *

###########################################################################
# Get the Score of the final ensemble
# ===================================

predictions = model.predict(X_test) > 0.5
# proba = model.predict_proba(X_test)

print("Accuracy score:", accuracy_score(y_test, predictions))
print("MCC:", matthews_corrcoef(y_test, predictions))
print("MCC:", auc(y_test, predictions))

Accuracy score: 0.6398988135941737
MCC: 0.2051787200920004


ValueError: x is neither increasing nor decreasing : [0. 0. 0. ... 1. 0. 0.].

In [9]:
cm = confusion_matrix(y_test, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=["Ignored","Matched"])
disp.plot()

ValueError: Classification metrics can't handle a mix of binary and continuous targets